<a href="https://colab.research.google.com/github/philippchistyakov/google_colab/blob/main/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_2%D1%85_%D1%82%D0%B8%D0%BA%D0%B5%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
!pip install --upgrade -q gspread_pandas
!pip install yfinance
!pip install  plotly==4.9.0

import pandas as pd
import yfinance as yf
import plotly.express as px

#@markdown ### start_date - Дата начала расчетов
start_date = '2020-09-01' #@param {type:"string"}
end_date = '2021-05-01' #@param {type:"string"}
#@markdown ### a и b - тикеты, в yahoo finance
#@markdown Например ссылка https://finance.yahoo.com/quote/BTC-USD/ , а тикет BTC-USD
a = 'ICLN' #@param {type:"string"}
b =  'FSLR' #@param {type:"string"}

test_yf =yf.download([a, b], start=start_date, end = end_date, group_by = 'ticker')

df = pd.DataFrame()
first_t = test_yf.columns.levels[0][0]
second_t = test_yf.columns.levels[0][1]
df[first_t]=test_yf[first_t]['Close']
df[second_t] = test_yf[second_t]['Close']
df = df.dropna()

result_df = pd.DataFrame()
for day_n in range(df.shape[0]):
  total_invest = 10000000000
  first_pos = (total_invest / df[first_t]).values[day_n]
  second_pos = (total_invest / df[second_t]).values[day_n]

  on_last_day_first = (df[first_t].values[-1] * first_pos - total_invest) / total_invest
  on_last_day_second = (df[second_t].values[-1] * second_pos - total_invest) / total_invest
  dttm = df.index[day_n]
  result_df = result_df.append(pd.DataFrame([dttm, on_last_day_first, on_last_day_second]).T)

result_df.columns = ['dt', f'{first_t} %', f'{second_t} %']
result_df.set_index('dt', inplace=True, drop=True)


fig = px.scatter(result_df,
             x=result_df.index,
             y=result_df.columns,
             marginal_y="violin",
             title=f"ПиУ% для {first_t} и {second_t}, Где по X дата входа в сделку, а по Y ПиУ % на {end_date} ",
             )
fig.update_layout(
    yaxis=dict(tickformat=',.2%'),
    xaxis2=dict(tickformat=',.2%')
)

fig.show()

[*********************100%***********************]  2 of 2 completed
